In [1]:
clean_up = True
%run stdPackages.ipynb

*Set up example:*

In [2]:
ws = gams.GamsWorkspace(working_directory=d['work'])
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(1,5)})
name = 'A'
data_str = os.path.join(d['data'],'Production_A.xlsx')
read_trees = {'Tree1': {'f':'CES'}, 'Tree2': {'f': 'MNL_out'}} # Keys refer sheet, f refers to type.
Tree = nestingTree.aggTree_from_data(data_str, read_trees = read_trees, name = name)() # apply call function.
P = CGE_Production.Production(tree = Tree, glob = glob)

# ```Production```

The ```Production``` module uses nesting trees to build a dynamic, multi-sector model with nested production functions. The fundamental setup is the following:

* The model consists of three fundamental sets that most equations/variables are defined over: (1) $n$ the set of goods in the economy, (2) $s$ the set of sectors, $t$ the time index. We generally use $nn$, $nnn$, $ss$ etc. as aliases for the fundamental sets.
* We refer to quantities, prices, and values as $(qS, qD, pS, pD, vS, vD)$. The $S$ indicates that the variable is on the supply of a good, the $D$ that it is a demand variable. Intermediate goods are also referred to as $qD[t,s,n]$.
* Market clearing ensures that the sum of supply equals the sum of supply, for a subset of goods $n$. As goods can be demanded/supplied by more than one sector, the market clearing condition is:
$$\begin{align}
    \sum_{s\in d\_qS[s,n]} qS[t,s,n] = \sum_{s\in d\_qD[s,n]} qD[t,s,n],
\end{align}$$
where $d\_qS$ and $d\_qD$ are dummies identifying which sectors are active for the relevant good.

## 1. Baseline mode:
In the baseline mode, the endogenous/exogenous variables of the module are:
* Input prices are taken as given. A subset ```input_A[s,n]``` identifies the inputs of the nesting tree (created automatically from the nesting trees). So, $pD[s,n]$ are exogenous for inputs (in partial equilibrium).
* Input quantities, intermediate quantities and prices, are all endogenous in the model. The subset ```int_A[s,n]``` identifies intermediate goods. So, $qD[s,n]$ are generally endogenous, so are $pD[s,n]$ for intermediate goods.
* Output prices are endogenous while output quantities are exogenous. The subset ```output[s,n]``` identifies outputs. The implicit assumption of the production module is that the entire tree exhibits constant returns to scale (linear homogeneity). So, we can only identify the price $pS[s,n]$, but not the quantity $qS[s,n]$ (in partial equilibrium).

In the example we loaded here, the inputs, intermediate goods, and outputs are given by:

In [3]:
P.get('input') # inputs

MultiIndex([('s1', 'K'),
            ('s1', 'M'),
            ('s2', 'K'),
            ('s1', 'L'),
            ('s2', 'M'),
            ('s2', 'L')],
           names=['s', 'n'])

In [4]:
P.get('int') # intermediate goods

MultiIndex([('s2',  'Y'),
            ('s1', 'KL')],
           names=['s', 'n'])

In [5]:
P.get('output') # outputs

MultiIndex([('s1', 'a'),
            ('s2', 'c'),
            ('s2', 'b')],
           names=['s', 'n'])

We can test running the model (we store a checkpoint to re-run from):

In [6]:
P.compile(initDB=True)
# P.s['solve'] = f"""@SolveEmptyNLP({P.s['name']})"""
P.write();
model = GmsPy.GmsModel(ws=ws, **{'cns':'CONOPT4'})
checkpoint = model.ws.add_checkpoint()
P.run(model=model, options_run = {'checkpoint': checkpoint})

### A small detour: Sneaky solve

Let's test the sneaky solve method: Change some of the exogenous variables / parameters in a loop. Extract some of the endogenous variables in the loop:

*Inputs for sneaky solve:*

In [7]:
# args: db0, dbT, name
db0 = model.out_db
dbT = GpyDB()
dbT['g_LR'] = 0.05
dbT['sigma'] = db0['sigma'].vals*2.1
dbT['pD'] = adj.rc_pd(db0['pD'], P.get('input'))[0:1] * 2
name = 'shock'
# kwargs with (mostly) default values used:
n = 10
extractSol = {'qD': P.g('int'), 'pD': ('and', [P.g('int'), P.g('t0')])}
db_name = 'grids'
loop = 'l1'
gridtype = 'linear'
phi= 1
checkDiff = True
error = 1e-11

Get database with grids:

In [8]:
db = auxFunctions.gridDB(db0,dbT,name, n = 10, extractSol = extractSol, db_name = db_name, loop=loop, gridtype=gridtype, phi = phi, checkDiff = checkDiff, error = error)

The database automatically creates:
* Loop index with name ```loop``` (pandas index, here 'l1').
* Subsets that reflect overlap in domains for variables/parameters in ```db0``` and ```dbT```. These are named ```x_name_ss``` with x being the symbol name and name referring to the database name. The 'ss' indicates that it is the relevant subset.
* Parameters with grids between symbols in ```db0``` and ```dbT```. These are named ```x_name```. The grids are of the type ```gridtype``` with ```n``` nodes.
* Solution parameters used to store solution when looping. These are named ```sol_x_name``` with x being the symbol name and name referring to the database name.

We use the grid database to write the relevant gams text by calling:

In [9]:
text = GmsWrite.loopUpdateSolve(loop, name, db, db0, db.updateDict, updateSolDict = db.updateSolDict, model = P.s['name'])

The entire thing can be fixed using (it uses all the settings outlined above as default):

In [10]:
text, db = GmsWrite.SolveLoop(P.s, dbT, **{'extractSol': extractSol})

To run the experiment, we add the database to the ```GmsModel``` and use the ```text``` as run file:

In [11]:
model.addDB(db)
model.run(run=text, options_add={'checkpoint':checkpoint})
model.out_db.get('sol_qD_shock').unstack('l1')

l1      l1_1 l1_10 l1_2 l1_3 l1_4 l1_5 l1_6 l1_7 l1_8 l1_9
t s  n                                                    
1 s1 KL  0.5   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  s2 Y   2.0   2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0
2 s1 KL  0.5   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  s2 Y   2.0   2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0
3 s1 KL  0.5   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  s2 Y   2.0   2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0
4 s1 KL  0.5   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  s2 Y   0.5   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5

### Sneaky solve test:

All the steps that we outlined above can be executed more simply using the ```sneakySolve``` method that is implemented for all ```GmsPython``` instances. This method uses all the same inputs as the basic ```run``` method, and adds two inputs: The target database (arg) and kwargs used in the ```SolveLoop``` method above. It returns a checkpoint that we can proceed from and the ```GmsModel``` instance:

In [12]:
model, cp = P.sneakySolve(dbT, ws = ws, loop_kwargs = {'extractSol': extractSol}, **{'cns': 'CONOPT4'})

## 2. Calibration mode:
The module includes a calibration mode, where the settings for exogenous/endogenous groups are flipped:
* All inputs and intermediate quantities in the baseline year ```(t0[t])``` are exogenous.
* To identify the quantities, we endogenize their counterparts in share parameters ```mu[s,n,nn]```.
* We add a couple of small adjustments if (they are added automatically):
    * Multiple output sectors: Only one output price is kept endogenous. All other output prices are exogenous. 
    * If there are nesting trees that are "scale preserving" (sum of quantity of branches = quantity of corresponding knot), there is an implicit quantity restriction that does not allow us to endogenize all share parameters in that nest. 

We switch to the calibration state and re-run:

In [13]:
model.run(run=P.s.writeSolveState('C'), options_add = {'checkpoint': checkpoint})

## 3. Adding durables:

When we add durables, we think of the following optimization problem:
$$\begin{align}
    \max_{\lbrace K_{s+1}, I_s\rbrace_{s=t}^T}&\sum_{s=t}^{T} \dfrac{1}{R_{t,s}}\left[p^K_sK_s-p^I_sI_s-\Psi\left(I_s,K_s\right)\right] \\ 
    \text{s.t. }K_{s+1} &= K_s(1-\delta)+I_s \\ 
    K_{T+1} &= K_T(1+\rho_K) \\
    K_t&\geq 0\text{ given}.
\end{align}$$

The Lagrangian is defined as:
$$\begin{align}
    \mathcal{L} = \sum_{s=t}^{T}\dfrac{1}{R_{t,s}}\left\lbrace p_s^KK_s-p_s^II_s-\Psi\left(I_s,K_s\right)+\lambda_s\left[I_s+K_s(1-\delta)-K_{s+1}\right]\right\rbrace 
\end{align}$$

The first order conditions are then:
$$\begin{align}
    \dfrac{\partial \mathcal{L}}{\partial K_s} &= -\lambda_s +\dfrac{R_{t,s}}{R_{t,s+1}} \left[p_{s+1}^K-\dfrac{\partial \Psi_{s+1}}{\partial K_{s+1}}+\lambda_{s+1}(1-\delta)\right] = 0 \\ 
    \dfrac{\partial \mathcal{L}}{\partial I_s} &= -p_s^I-\dfrac{\partial \Psi_s}{\partial I_s} + \lambda_s =0. 
\end{align}$$

Combining the first order conditions, optimality requires:
$$\begin{align}
    p_s^I+\dfrac{\partial \Psi_s}{\partial I_s} &= \dfrac{R_{t,s}}{R_{t,s+1}}\left[p_{s+1}^K-\dfrac{\partial \Psi_{s+1}}{\partial K_{s+1}}+(1-\delta)\left(p_{s+1}^I+\dfrac{\partial \Psi_{s+1}}{\partial I_{s+1}}\right)\right]
\end{align}$$

Rearranging this equation and using $R_{s+1}=R_{t,s+1}/R_{t,s}$ and quadratic installation costs, we then have the system of equations:
$$\begin{align}
    p_t^K &= R_t\left(p_{t-1}^I+\dfrac{\partial \Psi_{t-1}}{\partial I_{t-1}}\right)+\dfrac{\partial \Psi_t}{\partial K_t}-(1-\delta)\left(p_t^I+\dfrac{\partial \Psi_t}{\partial I_t}\right), \qquad \forall t_0<t<T, \tag{Q} \\ 
    K_{t+1}&=K_t(1-\delta)+I_t, \qquad \forall t<T, \tag{LOM} \\ 
    K_{T}  &=K_{T-1}(1+\rho_K), \tag{TVC} \\ 
    \Psi_t &= K_t\dfrac{\phi_1}{2}\left(\dfrac{I_t}{K_t}-\phi_2\right)^2, \qquad \forall t<T, \tag{InstCosts}
\end{align}$$

The optimality condition with quadratic costs are then given by:
$$\begin{align}
    p_t^K = R_t\left[p_{t-1}^I+\phi_1\left(\dfrac{I_{t-1}}{K_{t-1}}-\phi_2\right)\right]+\dfrac{\phi_1}{2}\left(\dfrac{I_t}{K_t}-\phi_2\right)^2-(1-\delta)\left[p_t^I+\phi_1\left(\dfrac{I_t}{K_t}-\phi_2\right)\right]
\end{align}$$

Assume that we have solved the model without durables first with $p_K = 1$. This is the stationary equilibrium if:
$$\begin{align}
    I_t &= \delta K_t \\
    \phi_2 &= \delta \\
    p_t^I &= \dfrac{p_t^K}{R_t-(1-\delta)}.
\end{align}$$

In the production module, we take the price on the investment goods ($I_t$) as given (in partial equilibrium). Note that the causality of the durables module is a bit complicated:
* For the very first period, we take the level $K_{t_0}$ as given. The standard nested production function setup includes an equation for the demand of $K_{t_0}$, however. Causally, this equation is used to identify the price $p^K_{t_0}$.
* For the other periods $t_0<t<T$, the quantity of $K_t$ is determined from the standard nested production function equations.
* The investment levels $I_t$ are determined from the law of motion (LOM).
* The quantity $K_T$ is determined from the (TVC).
* The prices on capital for $t_0<t<T$ are determined by the (Q) equation.
* The level of installation costs $\Psi$ from (InstCosts).

So, when we add these equations, we do the following:
* Adjust the subsets in the standard nested production tree: 
    * All durables are removed from the subset ```input```.
    * The corresponding investment goods are added to the subset ```input``` instead.
* Define new groups of variables:
    * Exogenous variables: Investment parameters $(\phi_1,\phi_2,\delta)$, interest rate $R_t$ (in partial equilibrium), and initial level of durables $(K_{t_0})$. Note that by adding investment variables to the set ```input```, they are automatically included in the correct endogenous/exogenous groups. 
    * Endogenous: Quantity of durables for $t>t_0$ and prices for $t<T$. Also, the level of installation costs per unit of output is computed.

Note that when the model is calibrated, the share parameter on $K_{t_0}$ is still endogenized, while the corresponding investment variable $I_{t_0}$ is made exogenous. **NB: If the durable good enters in a scale preserving input nest, the share parameter might have been chosen to be exogenous. This might be an issue in calibration mode.**

### Specifying durables

Return data to the baseline solution first:

In [14]:
P.s.db = db0

We add durables by specifying what subset of variables are durables, and include a mapping from durable to the relevant investment good. For instance:

In [15]:
dur = adj.rc_pd( P.get('input'), pd.Index(['K'],name='n')) # durables are all inputs called 'K'
dur

MultiIndex([('s2', 'K'),
            ('s1', 'K')],
           names=['s', 'n'])

In [16]:
dur2inv = pd.MultiIndex.from_frame(dur.to_frame(index=False).assign(nn=lambda x: 'I_'+x.n)) # Simply specify the relevant investment goods as 'I_'+ name of the durable
dur2inv

MultiIndex([('s2', 'K', 'I_K'),
            ('s1', 'K', 'I_K')],
           names=['s', 'n', 'nn'])

We add the durables using the ```self.addDurables``` method:

In [17]:
P.addDurables(dur=dur, dur2inv = dur2inv)

### Solve with durables

In [18]:
P.s.setstate('B')

For the dynamic model, the initial values are critical to be able to identify an initial solution. The easiest way to do this is to start from a steady state solution. This is done by calling ```initDurables``` here. Note, however, that this will alter some of the parameters. 

In [19]:
P.initDurables()
P.compile()
# P.s['solve'] = f"""@SolveEmptyNLP({P.s['name']})"""
P.write();
model = P.run(exportTo = d['work'], ws=ws,**{'cns': 'CONOPT4'})